## Второй бот для отправки уведомлений о встречах

In [ ]:
# pip install schedule

In [5]:
import sqlite3
import telebot
from telebot import types
import json
import random
from datetime import datetime
from time import sleep
import pandas as pd

import schedule 
# https://schedule.readthedocs.io/en/stable/reference.html 
# https://schedule.readthedocs.io/en/stable/examples.html
# https://schedule.readthedocs.io/en/stable/_modules/schedule.html#run_pending - код библиотеки


bot = telebot.TeleBot("6637092297:AAE7hUz7mBorW-gP9r_qL_x-3UdEWlPRBpU") # нужен токен вашего бота meetingreminderzi16_notify_bot


# Функция для отправки сообщения с инфой
def dop_send_stat(message, meet_time, meet_location, meet_name):
    bot.send_message(message.chat.id, f'Встреча "{meet_name}" началась в {meet_time}: \n{meet_location}')
    
    
# Функция формирования нового поля с нужным текстом для установки на расписание
def fun_meet_text_msg_new(row):
    meet_time = ""
    if len(row['meet_time'].split(':')[0]) < 2:
        meet_time = f"0{row['meet_time']}"
    else: 
        meet_time = row['meet_time']
#         print(f'meet_time = {meet_time}')
        
    
    if row['meet_schedule'] == 'daily on weekdays':
#         print(f"schedule.every().day.at('{meet_time}').do(dop_send_stat, message, '{row['meet_location']}')")
        return f"schedule.every().day.at('{meet_time}').do(dop_send_stat, message, '{row['meet_time']}', '{row['meet_location']}', '{row['meet_name']}')"

# Функция перекладывает данные из таблицы в датафрейм и формирует новое поле для установки на расписание    
def parse_table_meet_remind(message):
    # подключаем базу данных
    conn = sqlite3.connect('meeting_reminder.db')
    
    # курсор для работы с таблицами
    cursor = conn.cursor()
    
    df_meet_remind = pd.read_sql_query("SELECT * FROM meeting_reminder WHERE chat_id=={}".format(message.chat.id), conn)
    df_meet_remind['meet_text_msg_new'] = df_meet_remind.apply(fun_meet_text_msg_new, axis=1)

    lst_meet_text_msg_new = list(df_meet_remind['meet_text_msg_new'])
    return lst_meet_text_msg_new

 

    
    
# def set_schedule(df: DataFrame):
#     pass
    
            
# шедулер для отправки сообщений в чат
@bot.message_handler(func = lambda message: 1==1)
def send_remind_message(message):
    # Собираю все встречи на сегодня вношу их в расписание (думаю с 8:00 до 19:00 нужно делать это каждые 10 минут, в остальное время только в 07:00)

    while True:

        print(f'============================== {datetime.now()} =====================================')

        lst_meet_text_msg_new = parse_table_meet_remind(message)

        #Установка на расписание
        list_dags = []

        #Очистка расписания
        schedule.clear()

        #Задание нового расписания
        for schedule_text in lst_meet_text_msg_new:
    #             print(f'schedule_text = {schedule_text}')
            list_dags.append(eval(schedule_text))
        print(list_dags)

        i = 0
        #Запускаю отправлятор сообщений (каждые 10 минут пока что)
        while i < 2:
    #     while True:
    #         print(f'------------- i = {i}')
            try:
                schedule.run_pending()
            except:
                logger.exception('Scheduler Exception') 
            sleep(5)
            
            i += 1
            
       
          
        

In [6]:
df_meet_remind = pd.DataFrame()

bot.polling(none_stop=True)

============================== 2024-01-31 15:20:07.079546 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-01-31 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-01-31 15:22:00)]
============================== 2024-01-31 15:20:17.089289 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at

============================== 2024-01-31 15:21:57.563615 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-01-31 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-01-31 15:23:00)]
============================== 2024-01-31 15:22:07.655112 =====================================
[Every 1 day at

============================== 2024-01-31 15:23:37.812926 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:23:00), Every 1 day at 15:25:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:25', 'https:

============================== 2024-01-31 15:24:37.857460 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:23:00), Every 1 day at 15:25:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:25', 'https:

============================== 2024-01-31 15:25:28.045274 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:23:00), Every 1 day at 15:25:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:25', 'https:

...

============================== 2024-01-31 17:33:57.478889 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:23:00), Every 1 day at 15:25:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:25', 'https:

============================== 2024-01-31 17:34:47.528484 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:23:00), Every 1 day at 15:25:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:25', 'https:

============================== 2024-01-31 17:35:37.578905 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:23:00), Every 1 day at 15:25:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:25', 'https:

============================== 2024-01-31 17:36:27.629896 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:23:00), Every 1 day at 15:25:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:25', 'https:

============================== 2024-01-31 17:37:17.691782 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 15:23:00), Every 1 day at 15:25:00 do dop_send_stat(<telebot.types.Message object at 0x0000026585754950>, '15:25', 'https:

In [7]:
df_meet_remind

""


============================== 2024-01-31 14:29:17.071703 =====================================
------------- i = 0
============================== 2024-01-31 14:29:17.740275 =====================================
[Every 1 day at 08:00:00 do dop_send_stat(<telebot.types.Message object at 0x00000230A63AEB50>, '8:00', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-01 08:00:00), Every 1 day at 21:52:00 do dop_send_stat(<telebot.types.Message object at 0x00000230A63AEB50>, '21:52', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-01-31 21:52:00), Every 1 day at 21:53:00 do dop_send_stat(<telebot.types.Message object at 0x00000230A63AEB50>, '21:53', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-01-31 21:53:00), Every 1 day at 21:54:00 do dop_send_stat(<telebot.types.Message object at 0x00000230A63AEB50>, '21:54', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-01-31 21:54:0

In [ ]:
# подключаем базу данных
conn = sqlite3.connect('meeting_reminder.db')
# курсор для работы с таблицами
cursor = conn.cursor()

df_meet_remind = pd.read_sql_query("SELECT * FROM meeting_reminder", conn)
df_meet_remind['meet_text_msg_new'] = df_meet_remind.apply(fun_meet_text_msg_new, axis=1)

lst_meet_text_msg_new = list(df_meet_remind['meet_text_msg_new'])

 

In [ ]:
# time_meet = datetime.strptime('1900-01-01 22:25:00', '%Y-%m-%d %H:%M:%S').replace(year=1900, month=1, day=1, microsecond=0)
# time_meet

In [ ]:
# datenow = datetime.now().replace(year=1900, month=1, day=1, microsecond=0)
# datenow

In [ ]:
# time_meet-datenow

In [1]:
#Первая версия сырая совсем
import sqlite3
import telebot
from telebot import types
import json
import random
from datetime import datetime
from time import sleep


bot = telebot.TeleBot("6637092297:AAE7hUz7mBorW-gP9r_qL_x-3UdEWlPRBpU") # нужен токен вашего бота meetingreminderzi16_notify_bot

# шедулер для отправки сообщений в чат
@bot.message_handler(func = lambda message: 1==1)
def send_remind_message(message):
    # Собираю все встречи на сегодня в ношу их в расписание (думаю с 8:00 до 19:00 нужно делать это каждые 10 минут, в остальное время только в 07:00)
    while True:
        if datetime.now().hour == 7:
            
    
    # 
    
    
    time_meet = datetime.strptime('1900-01-01 14:28', '%Y-%m-%d %H:%M').replace(year=1900, month=1, day=1, microsecond=0)
    print(f'time_meet = {time_meet}')
    while True:
        time_diff = time_meet - datetime.now().replace(year=1900, month=1, day=1, microsecond=0)
        datenow = datetime.now().replace(year=1900, month=1, day=1, microsecond=0)
        print(time_meet - datetime.now().replace(year=1900, month=1, day=1, microsecond=0))
#         print(f'time_meet = {time_meet}')
        print(f'datenow={datenow}')
        print(f'time_diff={time_diff}')
        if time_diff.days >= 0 and time_diff.seconds >=0 and time_diff.seconds <= 120:
            text = f'Встреча начнется через: {str(time_diff)} секунд'
            break
        elif time_diff.days < 0:
            text = f'Встреча уже началась: {str(-time_diff)} секунд назад'
            break
            
        sleep(10)
#     text = 'ha'    
    bot.send_message(message.chat.id, text)
    print('отправили')

    


IndentationError: expected an indented block after 'if' statement on line 17 (3214300635.py, line 23)